In [1]:
import dash
from dash import html
from dash import dcc # Recommended import for dcc in newer Dash versions (works for older too)
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# --- 1. Load the dataset ---
# Assuming 'spacex_launch_dash.csv' is in the same directory as this script
try:
    spacex_df = pd.read_csv("spacex_launch_dash.csv")
except FileNotFoundError:
    print("Error: 'spacex_launch_dash.csv' not found. Please ensure it's in the same directory.")
    # Create a dummy dataframe for testing if file not found, or exit
    spacex_df = pd.DataFrame({
        'Launch Site': ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
        'class': [1, 0, 1, 1], # 1 for success, 0 for failure
        'Payload Mass (kg)': [2500, 3000, 4000, 1500],
        'Booster Version': ['B0001', 'B0002', 'B0003', 'B0004']
    })


max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# --- 2. Create the Dash application instance ---
app = dash.Dash(__name__)

# --- 3. Define the app layout ---
app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # TASK 1: Add a dropdown list to enable Launch Site selection
    # The default select value is for ALL sites
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'}
                 ] + [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
                 value='ALL',
                 placeholder='Select a Launch Site',
                 searchable=True,
                 style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'textAlignLast': 'center'}
                ),

    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    # If a specific launch site was selected, show the Success vs. Failed counts for the site
    html.Div(dcc.Graph(id='success-pie-chart')),

    html.Br(),

    html.P("Payload range (Kg):", style={'textAlign': 'center', 'font-size': 20}),
    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000, # Adjustable for payload range
                    marks={i: f'{i}' for i in range(0, 10001, 1000)}, # Labels for the slider points
                    value=[min_payload, max_payload]), # Default range

    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# --- 4. Define Callbacks ---

# Callback for the Pie Chart (TASK 2)
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class',
                     names='Launch Site',
                     title='Total Successful Launches By Site')
    else:
        # filter data for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        # Count successful (class=1) and failed (class=0) launches
        success_counts = filtered_df['class'].value_counts().reset_index()
        success_counts.columns = ['class', 'count'] # Rename columns for clarity
        fig = px.pie(success_counts, values='count',
                     names='class',
                     title=f'Total Success and Failure Launches for Site {entered_site}')
        # Map 0 to 'Failure' and 1 to 'Success' for better labels
        fig.update_traces(marker=dict(colors=['#EF553B', '#636EF7']), # Example colors for Failure/Success
                          labels=['Failure', 'Success'] if 0 in success_counts['class'].values else ['Success'])

    return fig

# Callback for the Scatter Chart (TASK 4)
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_chart(entered_site, payload_range):
    low, high = payload_range

    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                                (spacex_df['Payload Mass (kg)'] <= high)]
        title = 'Correlation between Payload and Success for All Sites'
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == entered_site) &
                                (spacex_df['Payload Mass (kg)'] >= low) &
                                (spacex_df['Payload Mass (kg)'] <= high)]
        title = f'Correlation between Payload and Success for Site {entered_site}'

    fig = px.scatter(filtered_df,
                     x='Payload Mass (kg)',
                     y='class',
                     color='Booster Version', # Color by booster for more insight
                     title=title,
                     labels={'class': 'Launch Outcome (1=Success, 0=Failure)'},
                     hover_data=['Launch Site'] # Show Launch Site on hover
                    )
    return fig

# --- 5. Run the app ---
if __name__ == '__main__':
    app.run(debug=True)